In [ ]:
import numpy as np
import librosa
import cv2
import moviepy.editor as mpy
from tqdm import tqdm
import os

class MusicVisualizer:
    def __init__(self, image_path, audio_path, output_path):
        self.image_path = image_path
        self.audio_path = audio_path
        self.output_path = output_path
        self.fps = 30
        self.max_blur = 80
        self.intro_duration = 3

    def load_audio(self):
        print("Loading audio...")
        self.y, self.sr = librosa.load(self.audio_path)
        self.duration = librosa.get_duration(y=self.y, sr=self.sr)
        self.n_frames = int(self.duration * self.fps)

    def analyze_audio(self):
        print("Analyzing audio...")
        hop_length = self.sr // self.fps
        S = librosa.feature.melspectrogram(y=self.y, sr=self.sr, n_mels=64, fmax=8000, hop_length=hop_length)
        self.S_dB = librosa.power_to_db(S, ref=np.max)
        self.onset_env = librosa.onset.onset_strength(y=self.y, sr=self.sr, hop_length=hop_length)
        self.pitches, _ = librosa.piptrack(y=self.y, sr=self.sr, hop_length=hop_length)
        self.max_onset = np.max(self.onset_env)

    def load_image(self):
        print("Loading image...")
        self.img = cv2.imread(self.image_path)
        self.img = cv2.cvtColor(self.img, cv2.COLOR_BGR2RGB)
        self.height, self.width, _ = self.img.shape

    def initialize_video(self):
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        self.video = cv2.VideoWriter('temp_output.mp4', fourcc, self.fps, (self.width, self.height))

    def create_shapes_and_elements(self):
        self.random_color = (np.random.randint(0, 256), np.random.randint(0, 256), np.random.randint(0, 256))
        self.shapes = [
            GeometricShape('block', self.width, self.height, 0, self.random_color),  # 左上
            GeometricShape('block', self.width, self.height, 1, self.random_color),  # 右上
            GeometricShape('block', self.width, self.height, 2, self.random_color),  # 左下
        ]
        self.dynamic_grid = DynamicGrid(self.width, self.height, self.random_color)
        self.dynamic_elements = [DynamicElement('line', self.width, self.height, self.random_color)]
        self.particles = create_particles(100, self.width, self.height)
        self.grid_particles = create_particles(200, self.width, self.height)
        self.frosted_blocks = create_frosted_glass_blocks(self.width, self.height, block_size_range=(50, 150), num_blocks=30)


    def generate_frames(self):
        prev_bars = np.zeros(64)
        zoom_factor = 1.0
        previous_frames = []
        intro_frames = int(self.intro_duration * self.fps)

        for frame_num in tqdm(range(self.n_frames), desc="Generating frames", unit="frame"):
            audio_idx = int(frame_num * len(self.y) / self.n_frames)
            chunk = self.y[audio_idx:audio_idx + self.sr // self.fps]

            spec_frame = self.S_dB[:, min(frame_num, self.S_dB.shape[1] - 1)]
            current_onset = self.onset_env[min(frame_num, len(self.onset_env) - 1)]
            energy = np.mean(np.abs(chunk)) * 10
            pitch = np.mean(self.pitches[:, frame_num])

            frame = self.img.copy()

            if frame_num < intro_frames:
                blur_amount = self.max_blur - int((frame_num / intro_frames) * self.max_blur)
                frame = cv2.GaussianBlur(frame, (blur_amount * 2 + 1, blur_amount * 2 + 1), 0)

            self.particles = update_particles(self.particles, self.width, self.height, energy)
            draw_particles(frame, self.particles, (255, 255, 255))

            frame = self.draw_spectrum_bars(frame, spec_frame, prev_bars)
            frame = apply_blur_effect(frame, energy)
            frame, zoom_factor = self.apply_zoom_effect(frame, current_onset, zoom_factor)
            frame, bounce_offset = apply_bounce_effect(frame, current_onset, self.max_onset, self.height)
            frame = apply_color_shift(frame, int(energy * 10))

            if np.random.rand() < 0.1:
                frame = apply_glitch_effect(frame)

            previous_frames = self.update_previous_frames(frame, previous_frames)
            frame = apply_trailing_effect(frame, previous_frames)

            for shape in self.shapes:
                shape.update(energy, current_onset, self.max_onset, pitch)
                frame = shape.draw(frame)

            for element in self.dynamic_elements:
                element.update(energy, current_onset, self.max_onset, pitch)
                frame = element.draw(frame)

            self.dynamic_grid.update(pitch)
            frame = self.dynamic_grid.draw(frame, self.grid_particles)  

            frame = apply_frosted_glass_effect(frame, self.frosted_blocks, blur_strength=85)

            self.video.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))

        self.video.release()

    def draw_spectrum_bars(self, frame, spec_frame, prev_bars):
        bar_width = int(self.width / (64 + 1.5))
        bar_spacing = int(bar_width / 2)
        max_bar_height = self.height // 2

        target_heights = np.interp(spec_frame, [self.S_dB.min(), self.S_dB.max()], [0, max_bar_height])
        prev_bars = prev_bars * 0.7 + target_heights * 0.3
        bar_heights = prev_bars.astype(int)

        overlay = frame.copy()

        for j, bar_height in enumerate(bar_heights):
            x1 = j * (bar_width + bar_spacing)
            x2 = x1 + bar_width

            cv2.rectangle(overlay, (x1, self.height // 2 - bar_height), (x2, self.height // 2), (*self.random_color, 100), -1)
            cv2.rectangle(overlay, (x1, self.height // 2), (x2, self.height // 2 + bar_height), (*self.random_color, 100), -1)
            cv2.rectangle(overlay, (x1, self.height // 2 - bar_height), (x2, self.height // 2 + bar_height), (*self.random_color, 255), 1)

        alpha = 0.7
        frame = cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0)
        return frame

    def apply_zoom_effect(self, frame, current_onset, zoom_factor):
        target_zoom = 1 + 0.05 * current_onset / self.max_onset
        zoom_factor = zoom_factor * 0.7 + target_zoom * 0.3
        scaled_frame = cv2.resize(frame, None, fx=zoom_factor, fy=zoom_factor)
        start_y = (scaled_frame.shape[0] - self.height) // 2
        start_x = (scaled_frame.shape[1] - self.width) // 2
        frame = scaled_frame[start_y:start_y + self.height, start_x:start_x + self.width]
        return frame, zoom_factor

    def update_previous_frames(self, frame, previous_frames):
        if len(previous_frames) > 5:
            previous_frames.pop(0)
        previous_frames.append(frame.copy())
        return previous_frames

    def combine_audio_and_video(self):
        print("Combining audio and video...")
        video = mpy.VideoFileClip('temp_output.mp4')
        audio = mpy.AudioFileClip(self.audio_path).set_duration(video.duration)
        final_video = video.set_audio(audio)
        final_video.write_videofile(self.output_path, codec="libx264", audio_codec="aac")

    def create_visualization(self):
        self.load_audio()
        self.analyze_audio()
        self.load_image()
        self.initialize_video()
        self.create_shapes_and_elements()
        self.generate_frames()
        self.combine_audio_and_video()

class GeometricShape:
    def __init__(self, shape_type, width, height, position_index, color):
        self.shape_type = shape_type
        self.width = max(1, width)
        self.height = max(1, height)
        self.opacity = np.random.uniform(0.5, 1)
        self.position_index = position_index
        self.color = color
        self.tetris_shapes = [
            [[1, 1, 1, 1]],  
            [[1, 1], [1, 1]],  
            [[1, 1, 1], [0, 1, 0]],  
            [[1, 1, 1], [1, 0, 0]],  
            [[1, 1, 1], [0, 0, 1]],  
            [[1, 1, 0], [0, 1, 1]],  
            [[0, 1, 1], [1, 1, 0]]   
        ]
        self.reset()

    def reset(self):
        if self.shape_type == 'block':
            self.shape = np.array(self.tetris_shapes[np.random.randint(0, len(self.tetris_shapes))])
            self.size = (self.shape.shape[1] * 20, self.shape.shape[0] * 120)  
            
            margin = self.width // 8  
            positions = [
                (margin, margin),  
                (self.width - self.size[0] - margin, margin),  
                (margin, self.height - self.size[1] - margin),  
            ]
            self.position = positions[self.position_index]
            
            self.bounce_offset = 0
            self.zoom_factor = 1.0
        self.bounce_speed = 0

    def update(self, energy, current_onset, max_onset, pitch):
        if self.shape_type == 'block':
            pitch_factor = min(pitch / 200, 1) if pitch > 0 else 0  
            self.bounce_speed = current_onset / max(max_onset, 1e-5) * 10  
            self.bounce_offset = int(np.sin(self.bounce_speed) * 10 * pitch_factor)  
            self.zoom_factor = 1 + energy * 0.3  

    def draw(self, frame):
        if self.shape_type == 'block':
            bounced_position = (self.position[0], min(max(0, self.position[1] + self.bounce_offset), self.height - 1))
            overlay = frame.copy()
            zoomed_size = (int(self.size[0] * self.zoom_factor), int(self.size[1] * self.zoom_factor))
            
            block_size = (int(zoomed_size[0] / self.shape.shape[1]), int(zoomed_size[1] / self.shape.shape[0]))
            
            for i in range(self.shape.shape[0]):
                for j in range(self.shape.shape[1]):
                    if self.shape[i][j] == 1:
                        block_pos = (
                            bounced_position[0] + j * block_size[0],
                            bounced_position[1] + i * block_size[1]
                        )
                        cv2.rectangle(overlay, block_pos, 
                                      (min(block_pos[0] + block_size[0], self.width - 1), 
                                       min(block_pos[1] + block_size[1], self.height - 1)), 
                                      self.color, -1)
            
            frame = cv2.addWeighted(overlay, self.opacity, frame, 1 - self.opacity, 0)
        return frame

class DynamicElement:
    def __init__(self, element_type, width, height, color, index=0):
        self.element_type = element_type
        self.width = width
        self.height = height
        self.index = index
        self.color = color
        self.thickness = 2 if element_type == 'line' else 10
        self.reset()

    def reset(self):
        if self.element_type == 'line':
            self.position = (self.width // 4, 0)

    def update(self, energy, current_onset, max_onset, pitch):
        if self.element_type == 'line':
            self.position = (self.width // 4 + int(np.sin(current_onset * 0.5) * 50), 0)

    def draw(self, frame):
        if self.element_type == 'line':
            cv2.line(frame, (self.position[0], 0), (self.position[0], self.height), self.color, self.thickness)
        return frame

class DynamicGrid:
    def __init__(self, width, height, color):
        self.width = width
        self.height = height
        self.grid_width = width // 2
        self.grid_height = height // 4
        self.grid_position = (width - self.grid_width, height - self.grid_height)
        self.cell_width = self.grid_width // 10
        self.cell_height = self.grid_height // 4
        self.line_thickness = 1
        self.grid_opacity = 0
        self.pitch_offset = 0
        self.color = color

    def update(self, pitch):
        pitch_factor = min(pitch / 100, 1) if pitch > 0 else 0
        self.cell_width = int(self.grid_width // 10 * (0.5 + 0.5 * pitch_factor))
        self.grid_opacity = min(1, pitch_factor * 2)
        self.pitch_offset = int(pitch_factor * self.grid_height // 2)

    def draw(self, frame, particles):
        overlay = frame.copy()
        for x in range(self.grid_position[0], self.width, self.cell_width):
            cv2.line(overlay, (x, self.grid_position[1] - self.pitch_offset), (x, self.height), self.color, self.line_thickness)
        for y in range(self.grid_position[1], self.height, self.cell_height):
            cv2.line(overlay, (self.grid_position[0], y), (self.width, y), self.color, self.line_thickness)

        for i in range(len(particles)):
            x, y, size, speed = particles[i]
            if self.grid_position[0] <= x <= self.width and self.grid_position[1] - self.pitch_offset <= y <= self.height:
                cv2.circle(overlay, (int(x), int(y)), int(size * 2), (255, 255, 255), -1)

        frame = cv2.addWeighted(overlay, self.grid_opacity, frame, 1 - self.grid_opacity, 0)
        return frame

def get_files(extensions):
    return [f for f in os.listdir('.') if f.lower().endswith(tuple(extensions))]

def select_file(file_type, extensions):
    files = get_files(extensions)

    if not files:
        print(f"No {file_type} files found in the current directory.")
        return None

    print(f"Available {file_type} files:")
    for i, file in enumerate(files, 1):
        print(f"{i}. {file}")

    while True:
        try:
            choice = int(input(f"Enter the number of the {file_type} file you want to use: "))
            if 1 <= choice <= len(files):
                return files[choice - 1]
            else:
                print("Invalid choice. Please try again.")
        except ValueError:
            print("Invalid input. Please enter a number.")

def select_mp3_file():
    return select_file("MP3", ['.mp3'])

def select_image_file():
    return select_file("image", ['.png', '.jpg', '.jpeg'])         

def create_shapes_and_elements(width, height):
    shapes = [GeometricShape('block', width, height) for _ in range(10)]
    shapes.append(GeometricShape('grid', width, height))
    dynamic_elements = [DynamicElement('line', width, height)] 
    return shapes, dynamic_elements

def create_particles(num_particles, width, height):
    particles = np.random.rand(num_particles, 4)
    particles[:, 0] *= width
    particles[:, 1] *= height
    particles[:, 2] = np.random.randint(2, 5, num_particles)
    particles[:, 3] = np.random.randint(1, 5, num_particles)
    return particles

def create_frosted_glass_blocks(width, height, block_size_range=(50, 150), num_blocks=10):
    blocks = []
    for _ in range(num_blocks):
        block_width = np.random.randint(block_size_range[0], block_size_range[1])
        block_height = np.random.randint(block_size_range[0], block_size_range[1])
        block_x = np.random.randint(0, width - block_width)
        block_y = np.random.randint(0, height - block_height)
        blocks.append((block_x, block_y, block_width, block_height))
    return blocks

def update_particles(particles, width, height, energy):
    particles[:, 1] -= particles[:, 3] * energy
    reset = particles[:, 1] < 0
    particles[reset, 1] = height
    particles[reset, 0] = np.random.randint(0, width, np.sum(reset))
    return particles

def draw_particles(frame, particles, color):
    for x, y, size, _ in particles:
        cv2.circle(frame, (int(x), int(y)), int(size), color, -1)

def apply_blur_effect(frame, energy):
    blur_amount = max(1, int(energy * 2))
    return cv2.GaussianBlur(frame, (blur_amount * 2 + 1, blur_amount * 2 + 1), 0)

def apply_bounce_effect(frame, current_onset, max_onset, original_height):
    bounce_amount = int(30 * current_onset / max_onset)
    if bounce_amount > 0:
        frame = frame[bounce_amount:-bounce_amount, :]
    return cv2.resize(frame, (frame.shape[1], original_height)), bounce_amount

def apply_frosted_glass_effect(frame, blocks, blur_strength=15):
    result = frame.copy()
    for x, y, w, h in blocks:
        roi = result[y:y+h, x:x+w]
        blurred = cv2.GaussianBlur(roi, (blur_strength, blur_strength), 0)
        mean = cv2.mean(blurred)[0]
        contrast_enhanced = cv2.addWeighted(blurred, 1.5, blurred, 0, mean * -0.5)
        edge_enhanced = cv2.addWeighted(contrast_enhanced, 1.5, cv2.Laplacian(contrast_enhanced, cv2.CV_8U), 0.5, 0)
        alpha = 0.7
        result[y:y+h, x:x+w] = cv2.addWeighted(roi, alpha, edge_enhanced, 1-alpha, 0)
    return result

def apply_glitch_effect(frame, strength=10):
    height, width, _ = frame.shape
    glitch_frame = frame.copy()
    num_slices = np.random.randint(1, strength)
    for _ in range(num_slices):
        slice_height = np.random.randint(1, height // strength)
        start_y = np.random.randint(0, height - slice_height)
        start_x = np.random.randint(-strength, strength)
        end_x = width + start_x
        if start_x > 0:
            end_x = min(end_x, width)
            glitch_frame[start_y:start_y + slice_height, start_x:end_x] = frame[start_y:start_y + slice_height, :end_x - start_x]
        else:
            start_x = max(start_x, -width)
            glitch_frame[start_y:start_y + slice_height, :end_x] = frame[start_y:start_y + slice_height, -start_x:]
    return glitch_frame

def apply_trailing_effect(frame, previous_frames):
    alpha = 0.7
    result = frame.copy()
    for prev_frame in previous_frames:
        result = cv2.addWeighted(result, alpha, prev_frame, 1 - alpha, 0)
    return result

def apply_color_shift(frame, shift_amount):
    b, g, r = cv2.split(frame)
    b = np.roll(b, shift_amount, axis=1)
    r = np.roll(r, -shift_amount, axis=1)
    return cv2.merge([b, g, r])

def main():
    image_path = select_image_file()
    audio_path = select_mp3_file()
    output_path = f"{audio_path.split('.')[0]}.mp4"
    print(f"Visualizing {audio_path} with {image_path}...")
    visualizer = MusicVisualizer(image_path, audio_path, output_path)
    visualizer.create_visualization()

if __name__ == "__main__":
    main()